<a href="https://colab.research.google.com/github/gabrielbonfim1/MTP5/blob/main/P5mt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Programa para auxiliar na solução da prova P5 da disciplina de máquinas térmicas, FEG - UNESP
##### - Feito por Gabriel Alves , Jan/2021 ano de Pandemia.

-> Importando bibliotecas necessárias para o programa


In [ ]:
import pandas as pd     # Biblioteca para importar as turbinas a gás
!pip install pyromat
import pyromat as pm    # Biblioteca de estados termodinâmicas

-> Importando o catálogo de turbinas a gás para a variável df (DataFrame)

In [6]:
df = pd.read_excel("https://github.com/gabrielbonfim1/MTP5/blob/main/TurbinasaGas.xlsx?raw=true", sheet_name="Planilha1")

-> Como todos os exercícios anteriores do ano, vamos assumir turbina a gás utilizando Gás natural, PCI = 50000 kJ/Kg , em paridade térmica.
-> Além disso assumiremos alguns valores usados previamente. Chequem antes das prova.

In [9]:
PCI = 50000
cpg = 1.2              #kJ/Kg   , Cp dos gases de exaustão.
Tchi = 150             #Celsius , temperatura da chaminé inicial.

-> Para utilização do programa será necessário escolher o estado de vapor vivo (temperatura e pressão), o estado de entrada na caldeira, e a vazão total necessária. Preencher os valores na célula a seguir.

In [ ]:
P_saida =     10        #Pressão de vapor na saida da caldeira de recuperação [MPa]
T_saida =     450       #Temperatura de saida da água na caldeira (Ta) [C]
P_entrada =   11.66     #Pressão de vapor na entrada da caldeira de recuperação [MPa]
T_entrada =   127       #Temperatura de entrada da água na caldeira (Ta) [C]
mv=          6.96       #Necessidade de vapor vivo a ser gerado [Kg/s]

## Código do Programa:

In [ ]:
hv=agua.h(T=T_saida, p=P_saida)
ha=agua.h(T=T_entrada, p=P_saida)
T_sat = agua.Ts(p=P_saida)
hls=agua.h(p=P_saida, x=0)
hvs=agua.h(p=P_saida, x=1)